# Remove unavailable videos and change names (uncommented)

In [ ]:
import json
import os
import cv2
from pathlib import Path

video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
json_file = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/WLASL_v0.3.json"

with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

video_labels = {}
for entry in data:
    gloss = entry["gloss"]
    for instance in entry["instances"]:
        video_id = str(instance["video_id"])  
        if video_id not in video_labels:
            video_labels[video_id] = []
        video_labels[video_id].append(gloss)

video_folder = Path(video_folder)

if not video_folder.exists():
    raise FileNotFoundError(f"Der Ordner {video_folder} wurde nicht gefunden.")

video_files = list(video_folder.glob("*"))


deleted_videos = []
renamed_videos = []

def is_video_valid(video_path):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return False  
    ret, _ = cap.read()
    cap.release()
    return ret 

for video_file in video_files:
    if not video_file.is_file():
        continue

    video_id = video_file.stem 

    if os.path.getsize(video_file) == 0 or not is_video_valid(video_file):
        deleted_videos.append(video_file.name)
        os.remove(video_file)
        continue

    # labels = video_labels.get(video_id, [])
    # if labels:
    #     new_name = f"{video_id}_{'_'.join(labels)}{video_file.suffix}"
    #     new_path = video_file.parent / new_name
    #     os.rename(video_file, new_path)
    #     renamed_videos.append((video_file.name, new_name))

print("Gelöschte Videos:", deleted_videos)
print("Umbenannte Videos:", renamed_videos)


# Add frame count to video names

In [ ]:
import cv2
from pathlib import Path

video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"

if not os.path.exists(video_folder):
    raise FileNotFoundError(f"Der Ordner {video_folder} wurde nicht gefunden.")

video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f))]

def get_frame_count(video_path):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return None 
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count

renamed_videos = []

for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    video_id, ext = os.path.splitext(video_file) 
    
    frame_count = get_frame_count(video_path)
    
    if frame_count is not None:
        new_name = f"{video_id}_{frame_count}{ext}"
        new_path = os.path.join(video_folder, new_name)
        os.rename(video_path, new_path)
        renamed_videos.append((video_file, new_name))

print("Umbenannte Videos mit Frames:", renamed_videos)

# Informations about videos

In [ ]:
import os
from collections import Counter

video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"

if not os.path.exists(video_folder):
    raise FileNotFoundError(f"Der Ordner {video_folder} wurde nicht gefunden.")

video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f))]

label_counter = Counter()

for video_file in video_files:
    filename, _ = os.path.splitext(video_file) 
    parts = filename.split("_") 

    if len(parts) == 3:  
        label = parts[1]  
        label_counter[label] += 1

print(f"Anzahl der verschiedenen Labels: {len(label_counter)}")
print("Top 10 häufigste Labels:")
for label, count in label_counter.most_common(10):
    print(f"{label}: {count}")


In [ ]:
# Groups per videos per label
from collections import Counter

label_distribution = Counter(label_counter.values())

print("Anzahl der Labels mit bestimmter Anzahl von Videos:")
for num_videos, num_labels in sorted(label_distribution.items()):
    print(f"{num_labels} Labels haben {num_videos} Videos")

# Delete videos with less than min_videos_per_label

In [ ]:
import os
from collections import Counter

# Directory containing videos
video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"

# Retrieve list of all videos
video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f))]

# Extract and count labels
label_counter = Counter()

for video_file in video_files:
    filename, _ = os.path.splitext(video_file)  # Remove file extension
    parts = filename.split("_")  # Split by "_"
    
    if len(parts) >= 3:  # Ensure correct format
        label = parts[1]  # Middle element is the label
        label_counter[label] += 1

# Minimum number of videos per label
min_videos_per_label = 8

# Delete files with too few videos
deleted_count = 0

for video_file in video_files:
    filename, _ = os.path.splitext(video_file)
    parts = filename.split("_")

    if len(parts) >= 3:
        label = parts[1]
        if label_counter[label] < min_videos_per_label:
            os.remove(os.path.join(video_folder, video_file))
            deleted_count += 1

print(f"{deleted_count} videos were deleted because their label had fewer than {min_videos_per_label} videos.")

---> start here

# Check Frames again

In [ ]:
import os
import cv2
import re

# Directory containing videos
video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"

if not os.path.exists(video_folder):
    raise FileNotFoundError(f"❌ The folder {video_folder} was not found.")

# Function to determine the current frame count
def get_frame_count(video_path):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return None 
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count

# Regex to find the frame count in the filename (e.g., `00624_accident_109_bright.mp4`)
frame_pattern = re.compile(r"_(\d+)(?:_[a-zA-Z]+)?\.mp4$")

renamed_videos = []

for video_file in os.listdir(video_folder):
    if not video_file.endswith(".mp4"):
        continue

    video_path = os.path.join(video_folder, video_file)

    # Extract the actual frame count
    frame_count = get_frame_count(video_path)
    if frame_count is None:
        print(f"⚠️ Warning: Could not determine frames for {video_file}. Skipping...")
        continue

    # Check if a frame count already exists in the name
    match = frame_pattern.search(video_file)
    
    if match:
        existing_frame_count = int(match.group(1))
        
        # Rename if the frame count does not match
        if existing_frame_count != frame_count:
            new_name = re.sub(frame_pattern, f"_{frame_count}.mp4", video_file)
            new_path = os.path.join(video_folder, new_name)
            os.rename(video_path, new_path)
            renamed_videos.append((video_file, new_name))
            print(f"🔄 {video_file} → {new_name} (Frames updated)")
    else:
        # If no frame count is present, add it
        name_parts = video_file.rsplit(".", 1)
        new_name = f"{name_parts[0]}_{frame_count}.mp4"
        new_path = os.path.join(video_folder, new_name)
        os.rename(video_path, new_path)
        renamed_videos.append((video_file, new_name))
        print(f"➕ {video_file} → {new_name} (Frames added)")

print("\n✅ All frame counts have been checked and updated if necessary!")

# Print videos with less than 40 min_videos_per_label

In [ ]:
import os
from collections import Counter

# Directories containing videos
raw_video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
augmented_video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"

# Count labels
label_counter = Counter()

for folder in [raw_video_folder, augmented_video_folder]:
    for video_file in os.listdir(folder):
        if not video_file.endswith(".mp4"):
            continue

        parts = video_file.split("_")
        if len(parts) >= 3:  # Ensure the name is correct
            label = parts[1]  # The second element is now the label
            label_counter[label] += 1

# Sort labels by number of videos (descending order)
sorted_labels = sorted(label_counter.items(), key=lambda x: x[1], reverse=True)

# Number of available labels
print(f"Number of remaining labels: {len(sorted_labels)}")

# Display top 10 most common labels
print("\nTop 10 most common labels:")
for label, count in sorted_labels[:10]:
    print(f"{label}: {count}")

# Display all labels with the number of videos
print("\nNumber of videos per label:")
for label, count in sorted_labels:
    print(f"{label}: {count}")

# Calculate maximum number of frames

In [ ]:
import os
import torch

# Directories containing videos
original_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
augmented_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"

# Determine the maximum number of frames across both folders
max_frames = 0

for folder in [original_folder, augmented_folder]:
    for video_file in os.listdir(folder):
        if not video_file.endswith(".mp4"):
            continue

        parts = video_file.replace(".mp4", "").split("_")
        
        if len(parts) >= 3:  # Ensure enough parts are present
            try:
                frame_count = int(parts[2])  # Frame count is at position 2
                max_frames = max(max_frames, frame_count)
            except ValueError:
                print(f"⚠️ Warning: Could not extract frame count from {video_file}")

print(f"✅ Maximum number of frames (across all videos): {max_frames}")

# Reduce number of videos per label

In [ ]:
import os
import random
from collections import Counter

# Directories containing videos
original_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
augmented_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"

# Target number of videos per label
target_videos_per_label = 36

# Count the number of videos per label
label_counter = Counter()

for folder in [original_folder, augmented_folder]:
    for video_file in os.listdir(folder):
        parts = video_file.split("_")
        if len(parts) >= 3:
            label = parts[1]  # The label is the second element
            label_counter[label] += 1

# Check if labels have fewer than 36 videos
for label, count in label_counter.items():
    if count < target_videos_per_label:
        print(f"⚠️ WARNING: Label '{label}' has only {count} videos and cannot be increased to 36!")

# Remove excess augmented videos randomly
for label, count in label_counter.items():
    if count > target_videos_per_label:
        # Find all augmented videos for this label
        label_videos = [f for f in os.listdir(augmented_folder) if f.split("_")[1] == label]

        # Warn if there are not enough augmented videos to delete
        if len(label_videos) < (count - target_videos_per_label):
            print(f"⚠️ WARNING: Not enough augmented videos to delete for label '{label}'!")

        # Randomly select videos to delete (only augmented ones)
        remove_videos = random.sample(label_videos, count - target_videos_per_label)

        # Delete the videos
        for video_file in remove_videos:
            os.remove(os.path.join(augmented_folder, video_file))
            print(f"🗑️ Deleted: {video_file}")

print("✅ All labels now have exactly 36 videos!")

# Normalize frames

In [ ]:
import os
import cv2
import torch
import numpy as np

# Directories
original_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
augmented_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"
processed_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_processed"
os.makedirs(processed_folder, exist_ok=True)

# Determine the maximum number of frames if not defined
if "max_frames" not in locals():
    max_frames = 195  # Default value if not previously calculated

# Function to extract frames as Torch tensors
def extract_frames(video_path, device="cuda"):
    cap = cv2.VideoCapture(video_path)
    frames = []

    if not cap.isOpened():
        print(f"⚠️ Warning: Could not open video: {video_path}")
        return torch.zeros((1, 3, 224, 224), dtype=torch.float32, device=device)  # Dummy frame
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))  # Resize frames to uniform size
        frame = torch.tensor(frame, dtype=torch.float32, device=device).permute(2, 0, 1)  # [H, W, C] → [C, H, W]
        frames.append(frame)

    cap.release()

    if not frames:
        print(f"⚠️ Warning: No frames extracted for {video_path}")
        return torch.zeros((1, 3, 224, 224), dtype=torch.float32, device=device)  # If empty, return dummy frame

    return torch.stack(frames, dim=0)

# Function for padding with GPU support
def pad_frames(frames, target_length, device="cuda"):
    num_frames = frames.shape[0]

    if num_frames < target_length:
        padding = torch.zeros((target_length - num_frames, 3, 224, 224), dtype=torch.float32, device=device)
        return torch.cat((frames, padding), dim=0)
    else:
        return frames[:target_length]  # Trim if too long

# Process all videos (Original + Augmented)
device = "cuda" if torch.cuda.is_available() else "cpu"

for folder in [original_folder, augmented_folder]:
    for video_file in os.listdir(folder):
        if not video_file.endswith(".mp4"):
            continue

        video_path = os.path.join(folder, video_file)

        try:
            # Extract frames and load to GPU
            frames = extract_frames(video_path, device=device)

            # Pad/Trim to `max_frames`
            padded_frames = pad_frames(frames, max_frames, device=device)

            # Save as `.npy` file (convert back to CPU)
            npy_path = os.path.join(processed_folder, video_file.replace(".mp4", ".npy"))
            np.save(npy_path, padded_frames.cpu().numpy())

            print(f"✅ {video_file} processed and saved as {npy_path}")

        except Exception as e:
            print(f"❌ Error processing {video_file}: {e}")

print("🚀 GPU-based normalization completed!")